LGBM 01 con linear_tree = true

lags 1 al 36 (3 años)

cantidad de clientes que compraron un producto

meses de vida 

In [1]:
# 📦 Importar librerías necesarias
import pandas as pd
import numpy as np
import random

# 🎲 Configurar semillas para reproductibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

print("✅ Librerías importadas correctamente")
print(f"🎲 Semilla configurada: {SEED} para reproductibilidad total")

✅ Librerías importadas correctamente
🎲 Semilla configurada: 42 para reproductibilidad total


In [2]:
# 📄 Cargar todos los datasets
print("Cargando datasets...")

# Load the sales data (tab-delimited)
sales = pd.read_csv("datasets/sell-in.txt", sep="\t", dtype={"periodo": str})
print(f"✅ Sales data cargado: {sales.shape}")

# Load the stocks data (tab-delimited) 
stocks = pd.read_csv("datasets/tb_stocks.txt", sep="\t", dtype={"periodo": str})
print(f"✅ Stocks data cargado: {stocks.shape}")

# Load the product information data (tab-delimited)
product_info = pd.read_csv("datasets/tb_productos.txt", sep="\t")
print(f"✅ Product info cargado: {product_info.shape}")

# Carga productos a predecir
product_predict = pd.read_csv("datasets/product_id_apredecir201912.txt", sep="\t", header=0)
print(f"✅ Productos a predecir cargados: {product_predict.shape}")

print("\n🎯 Todos los datasets cargados exitosamente")

Cargando datasets...
✅ Sales data cargado: (2945818, 7)
✅ Stocks data cargado: (13691, 3)
✅ Product info cargado: (1251, 7)
✅ Productos a predecir cargados: (780, 1)

🎯 Todos los datasets cargados exitosamente


In [3]:
# 🔍 Explorar estructura de los datos
print("EXPLORACIÓN DE DATOS")
print("="*50)

print("\n📊 SALES DATA:")
print(f"Columnas: {list(sales.columns)}")
print(f"Períodos únicos: {sales['periodo'].nunique()}")
print(f"Productos únicos: {sales['product_id'].nunique()}")
print("Primeras filas:")
print(sales.head())

print("\n📦 STOCKS DATA:")
print(f"Columnas: {list(stocks.columns)}")
print(f"Períodos únicos: {stocks['periodo'].nunique()}")
print(f"Productos únicos: {stocks['product_id'].nunique()}")
print("Primeras filas:")
print(stocks.head())

print("\n🏷️ PRODUCT INFO:")
print(f"Columnas: {list(product_info.columns)}")
print(f"Productos únicos: {product_info['product_id'].nunique()}")
print("Primeras filas:")
print(product_info.head())

print("\n🎯 PRODUCTOS A PREDECIR:")
print(f"Columnas: {list(product_predict.columns)}")
print(f"Total productos a predecir: {len(product_predict)}")
print("Primeras filas:")
print(product_predict.head())

EXPLORACIÓN DE DATOS

📊 SALES DATA:
Columnas: ['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn']
Períodos únicos: 36
Productos únicos: 1233
Primeras filas:
  periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0  201701        10234       20524                      0                 2   
1  201701        10032       20524                      0                 1   
2  201701        10217       20524                      0                 1   
3  201701        10125       20524                      0                 1   
4  201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  

📦 STOCKS DATA:
Columnas: ['periodo', 'product_id', 'stock_final']
Períodos únicos: 15
Productos únicos: 1095
Primeras filas:
  p

In [4]:
# 🔗 Verificar consistencia entre datasets
print("VERIFICACIÓN DE CONSISTENCIA")
print("="*50)

# Productos únicos en cada dataset
productos_sales = set(sales['product_id'].unique())
productos_stocks = set(stocks['product_id'].unique())
productos_info = set(product_info['product_id'].unique())

# Si product_predict tiene columna product_id
if 'product_id' in product_predict.columns:
    productos_predict = set(product_predict['product_id'].unique())
else:
    # Si la primera columna contiene los product_ids
    primera_columna = product_predict.columns[0]
    productos_predict = set(product_predict[primera_columna].unique())
    print(f"⚠️ Usando columna '{primera_columna}' como product_id")

print(f"📊 Productos en sales: {len(productos_sales)}")
print(f"📦 Productos en stocks: {len(productos_stocks)}")
print(f"🏷️ Productos en product_info: {len(productos_info)}")
print(f"🎯 Productos a predecir: {len(productos_predict)}")

# Verificar intersecciones
print(f"\n🔍 INTERSECCIONES:")
print(f"Sales ∩ Stocks: {len(productos_sales & productos_stocks)}")
print(f"Sales ∩ Product_info: {len(productos_sales & productos_info)}")
print(f"Sales ∩ Productos_predict: {len(productos_sales & productos_predict)}")
print(f"Stocks ∩ Productos_predict: {len(productos_stocks & productos_predict)}")
print(f"Product_info ∩ Productos_predict: {len(productos_info & productos_predict)}")

# Verificar rangos de fechas
print(f"\n📅 RANGOS DE FECHAS:")
print(f"Sales - períodos: {sales['periodo'].min()} a {sales['periodo'].max()}")
print(f"Stocks - períodos: {stocks['periodo'].min()} a {stocks['periodo'].max()}")

VERIFICACIÓN DE CONSISTENCIA
📊 Productos en sales: 1233
📦 Productos en stocks: 1095
🏷️ Productos en product_info: 1251
🎯 Productos a predecir: 780

🔍 INTERSECCIONES:
Sales ∩ Stocks: 1095
Sales ∩ Product_info: 1188
Sales ∩ Productos_predict: 780
Stocks ∩ Productos_predict: 779
Product_info ∩ Productos_predict: 780

📅 RANGOS DE FECHAS:
Sales - períodos: 201701 a 201912
Stocks - períodos: 201810 a 201912


In [5]:
# 📦 Instalar e importar LightGBM, Optuna y librerías adicionales
# %pip install lightgbm optuna

import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 🎲 Configurar semillas adicionales para reproductibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Configurar semilla para matplotlib (si se usa)
plt.rcParams['figure.max_open_warning'] = 0

print("✅ LightGBM, Optuna y librerías ML importadas correctamente")
print(f"🎲 Todas las semillas configuradas con SEED={SEED} para reproductibilidad")

✅ LightGBM, Optuna y librerías ML importadas correctamente
🎲 Todas las semillas configuradas con SEED=42 para reproductibilidad


In [6]:
# 🧹 Preparación de datos para el modelo LightGBM
print("PREPARACIÓN DE DATOS PARA LGBM - GRANULARIDAD POR PRODUCTO")
print("="*60)

# Convertir período a datetime para facilitar manipulación
sales['fecha'] = pd.to_datetime(sales['periodo'], format='%Y%m')
stocks['fecha'] = pd.to_datetime(stocks['periodo'], format='%Y%m')

# Filtrar solo productos que necesitamos predecir
if 'product_id' in product_predict.columns:
    productos_objetivo = product_predict['product_id'].tolist()
else:
    productos_objetivo = product_predict[product_predict.columns[0]].tolist()

print(f"🎯 Productos objetivo: {len(productos_objetivo)}")

# Filtrar sales y stocks para productos objetivo
sales_filtered = sales[sales['product_id'].isin(productos_objetivo)].copy()
stocks_filtered = stocks[stocks['product_id'].isin(productos_objetivo)].copy()

print(f"📊 Sales filtradas: {sales_filtered.shape}")
print(f"📦 Stocks filtradas: {stocks_filtered.shape}")

# AGREGACIÓN POR PRODUCTO: Sumar por producto y período (agregando todos los clientes)
sales_agg = sales_filtered.groupby(['product_id', 'fecha', 'periodo']).agg({
    'tn': 'sum',                    # Total toneladas por producto
    'customer_id': 'nunique',       # Número de clientes únicos
    'cust_request_qty': 'sum',      # Total cantidad solicitada
    'cust_request_tn': 'sum'        # Total toneladas solicitadas
}).reset_index()

# Renombrar columnas para claridad
sales_agg.rename(columns={
    'customer_id': 'num_customers',
    'cust_request_qty': 'total_request_qty', 
    'cust_request_tn': 'total_request_tn'
}, inplace=True)

print(f"📈 Sales agregadas por producto: {sales_agg.shape}")
print("Primeras filas de sales agregadas:")
print(sales_agg.head())

print(f"\n📊 Estadísticas por producto:")
print(f"  Promedio tn por producto-período: {sales_agg['tn'].mean():.2f}")
print(f"  Promedio clientes por producto-período: {sales_agg['num_customers'].mean():.2f}")
print(f"  Productos únicos: {sales_agg['product_id'].nunique()}")
print(f"  Períodos únicos: {sales_agg['periodo'].nunique()}")

PREPARACIÓN DE DATOS PARA LGBM - GRANULARIDAD POR PRODUCTO
🎯 Productos objetivo: 780
📊 Sales filtradas: (2293481, 8)
📦 Stocks filtradas: (10727, 4)
📈 Sales agregadas por producto: (22349, 7)
Primeras filas de sales agregadas:
   product_id      fecha periodo          tn  num_customers  \
0       20001 2017-01-01  201701   934.77222            186   
1       20001 2017-02-01  201702   798.01620            185   
2       20001 2017-03-01  201703  1303.35771            188   
3       20001 2017-04-01  201704  1069.96130            104   
4       20001 2017-05-01  201705  1502.20132            238   

   total_request_qty  total_request_tn  
0                479         937.72717  
1                432         833.72187  
2                509        1330.74697  
3                279        1132.94430  
4                701        1550.68936  

📊 Estadísticas por producto:
  Promedio tn por producto-período: 50.23
  Promedio clientes por producto-período: 102.62
  Productos únicos: 780
  Pe

In [7]:
# 🔧 Crear features de lag y combinar con stocks - GRANULARIDAD POR PRODUCTO
print("CREACIÓN DE FEATURES POR PRODUCTO")
print("="*50)

# Crear features de lag para cada producto
def create_lag_features(df, product_col, value_col, date_col, lags=[1, 2, 3, 6, 12]):
    """Crear features de lag para series temporales por producto"""
    df_features = df.copy()
    df_features = df_features.sort_values([product_col, date_col])
    
    for lag in lags:
        df_features[f'{value_col}_lag_{lag}'] = df_features.groupby(product_col)[value_col].shift(lag)
    
    return df_features

# Crear lags para ventas (tn) por producto - EXPANDIDO HASTA LAG 36
sales_with_lags = create_lag_features(
    sales_agg, 
    'product_id', 
    'tn', 
    'fecha', 
    lags=list(range(1, 37))  # Todos los lags de 1 a 36 meses (3 años completos)
)

# Crear lags para número de clientes por producto
sales_with_lags = create_lag_features(
    sales_with_lags, 
    'product_id', 
    'num_customers', 
    'fecha', 
    lags=[1, 2, 3]
)

# Crear lags para solicitudes de clientes
sales_with_lags = create_lag_features(
    sales_with_lags, 
    'product_id', 
    'total_request_tn', 
    'fecha', 
    lags=[1, 2, 3]
)

print(f"📊 Sales con lags por producto: {sales_with_lags.shape}")

# Agregar datos de stock por producto
stocks_agg = stocks_filtered.groupby(['product_id', 'fecha', 'periodo']).agg({
    'stock_final': ['mean', 'sum', 'std']  # Stock promedio, total y desviación estándar por producto
}).reset_index()

# Aplanar columnas multinivel
stocks_agg.columns = ['product_id', 'fecha', 'periodo', 'stock_tn_mean', 'stock_tn_sum', 'stock_tn_std']
stocks_agg['stock_tn_std'] = stocks_agg['stock_tn_std'].fillna(0)  # Rellenar NaN en std

# Combinar sales y stocks por producto
data_combined = pd.merge(
    sales_with_lags, 
    stocks_agg[['product_id', 'fecha', 'stock_tn_mean', 'stock_tn_sum', 'stock_tn_std']], 
    on=['product_id', 'fecha'], 
    how='left'
)

# Crear lags para stock por producto
data_combined = create_lag_features(
    data_combined, 
    'product_id', 
    'stock_tn_mean', 
    'fecha', 
    lags=[1, 2, 3]
)

data_combined = create_lag_features(
    data_combined, 
    'product_id', 
    'stock_tn_sum', 
    'fecha', 
    lags=[1, 2]
)

print(f"📦 Datos combinados con stock por producto: {data_combined.shape}")
print(f"Columnas disponibles: {len(data_combined.columns)}")
print("\nPrimeras columnas:")
print(data_combined.columns.tolist()[:15])
print("Últimas columnas:")
print(data_combined.columns.tolist()[-10:])

CREACIÓN DE FEATURES POR PRODUCTO
📊 Sales con lags por producto: (22349, 49)
📦 Datos combinados con stock por producto: (22349, 57)
Columnas disponibles: 57

Primeras columnas:
['product_id', 'fecha', 'periodo', 'tn', 'num_customers', 'total_request_qty', 'total_request_tn', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6', 'tn_lag_7', 'tn_lag_8']
Últimas columnas:
['total_request_tn_lag_2', 'total_request_tn_lag_3', 'stock_tn_mean', 'stock_tn_sum', 'stock_tn_std', 'stock_tn_mean_lag_1', 'stock_tn_mean_lag_2', 'stock_tn_mean_lag_3', 'stock_tn_sum_lag_1', 'stock_tn_sum_lag_2']


In [8]:
# 🎯 Crear target y features adicionales
print("CREACIÓN DE TARGET Y FEATURES ADICIONALES")
print("="*50)

# Crear target: tn de 2 períodos a futuro
data_combined = data_combined.sort_values(['product_id', 'fecha'])
data_combined['target'] = data_combined.groupby('product_id')['tn'].shift(-2)

# Crear features temporales
data_combined['mes'] = data_combined['fecha'].dt.month
data_combined['trimestre'] = data_combined['fecha'].dt.quarter
data_combined['año'] = data_combined['fecha'].dt.year

# 🔢 Crear feature de clientes únicos por producto
print(f"\n🔍 Creando feature de clientes únicos por producto...")
# Calcular cantidad total de clientes únicos que compraron cada producto
clientes_unicos_por_producto = sales_filtered.groupby('product_id')['customer_id'].nunique().reset_index()
clientes_unicos_por_producto.rename(columns={'customer_id': 'cant_clientes_unicos'}, inplace=True)

# Agregar esta información a data_combined
data_combined = pd.merge(
    data_combined,
    clientes_unicos_por_producto,
    on='product_id',
    how='left'
)

print(f"✅ Feature 'cant_clientes_unicos' agregada")
print(f"📊 Estadísticas de clientes únicos por producto:")
print(f"   Promedio: {data_combined['cant_clientes_unicos'].mean():.1f}")
print(f"   Mediana:  {data_combined['cant_clientes_unicos'].median():.1f}")
print(f"   Mínimo:   {data_combined['cant_clientes_unicos'].min()}")
print(f"   Máximo:   {data_combined['cant_clientes_unicos'].max()}")

# 🗓️ Crear feature de meses de vida por producto
print(f"\n🔍 Creando feature de meses de vida por producto...")
# Calcular primer y último período de compra para cada producto
vida_producto = sales_filtered.groupby('product_id')['fecha'].agg(['min', 'max']).reset_index()
vida_producto.columns = ['product_id', 'primer_periodo', 'ultimo_periodo']

# Calcular diferencia en meses
vida_producto['meses_vida'] = ((vida_producto['ultimo_periodo'] - vida_producto['primer_periodo']).dt.days / 30.44).round().astype(int)

# Asegurar que el mínimo sea 1 (productos con una sola compra tienen 1 mes de vida)
vida_producto['meses_vida'] = vida_producto['meses_vida'].apply(lambda x: max(1, x))

# Agregar esta información a data_combined
data_combined = pd.merge(
    data_combined,
    vida_producto[['product_id', 'meses_vida']],
    on='product_id',
    how='left'
)

print(f"✅ Feature 'meses_vida' agregada")
print(f"📊 Estadísticas de meses de vida por producto:")
print(f"   Promedio: {data_combined['meses_vida'].mean():.1f} meses")
print(f"   Mediana:  {data_combined['meses_vida'].median():.1f} meses")
print(f"   Mínimo:   {data_combined['meses_vida'].min()} meses")
print(f"   Máximo:   {data_combined['meses_vida'].max()} meses")
print(f"   Productos con > 24 meses: {(data_combined['meses_vida'] > 24).sum()}")
print(f"   Productos con > 36 meses: {(data_combined['meses_vida'] > 36).sum()}")

# Crear features estadísticas móviles
def create_rolling_features(df, product_col, value_col, date_col, windows=[3, 6, 12]):
    """Crear features de ventanas móviles"""
    df = df.sort_values([product_col, date_col])
    
    for window in windows:
        df[f'{value_col}_rolling_mean_{window}'] = df.groupby(product_col)[value_col].rolling(window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'{value_col}_rolling_std_{window}'] = df.groupby(product_col)[value_col].rolling(window, min_periods=1).std().reset_index(level=0, drop=True)
    
    return df

# Crear rolling features para ventas
data_combined = create_rolling_features(data_combined, 'product_id', 'tn', 'fecha', windows=[3, 6])

# 📊 Verificar features de lags de toneladas creados
print(f"\n📈 FEATURES DE LAGS DE TONELADAS DISPONIBLES:")
tn_lag_features = [col for col in data_combined.columns if col.startswith('tn_lag_')]
tn_lag_features.sort(key=lambda x: int(x.split('_')[-1]))  # Ordenar por número de lag
print(f"   Total de lags de toneladas: {len(tn_lag_features)}")
for i, feature in enumerate(tn_lag_features, 1):
    lag_num = feature.split('_')[-1]
    print(f"   {i:2d}. {feature} (toneladas de {lag_num} mes{'es' if int(lag_num) > 1 else ''} atrás)")

print(f"\n✅ Features expandidas: Lags de toneladas desde 1 hasta 36 meses")
print(f"🎯 Esto permite al modelo capturar patrones estacionales multi-anuales")

# Agregar información de productos si está disponible
if len(product_info) > 0:
    data_combined = pd.merge(
        data_combined, 
        product_info, 
        on='product_id', 
        how='left'
    )
    print(f"✅ Información de productos agregada")

print(f"📊 Dataset final: {data_combined.shape}")
print(f"📈 Registros con target válido: {data_combined['target'].notna().sum()}")

# Mostrar algunas estadísticas del target
target_stats = data_combined['target'].describe()
print(f"\n📊 Estadísticas del target:")
print(target_stats)

CREACIÓN DE TARGET Y FEATURES ADICIONALES

🔍 Creando feature de clientes únicos por producto...
✅ Feature 'cant_clientes_unicos' agregada
📊 Estadísticas de clientes únicos por producto:
   Promedio: 359.6
   Mediana:  372.0
   Mínimo:   22
   Máximo:   521

🔍 Creando feature de meses de vida por producto...
✅ Feature 'meses_vida' agregada
📊 Estadísticas de meses de vida por producto:
   Promedio: 31.9 meses
   Mediana:  35.0 meses
   Mínimo:   3 meses
   Máximo:   35 meses
   Productos con > 24 meses: 19798
   Productos con > 36 meses: 0

📈 FEATURES DE LAGS DE TONELADAS DISPONIBLES:
   Total de lags de toneladas: 36
    1. tn_lag_1 (toneladas de 1 mes atrás)
    2. tn_lag_2 (toneladas de 2 meses atrás)
    3. tn_lag_3 (toneladas de 3 meses atrás)
    4. tn_lag_4 (toneladas de 4 meses atrás)
    5. tn_lag_5 (toneladas de 5 meses atrás)
    6. tn_lag_6 (toneladas de 6 meses atrás)
    7. tn_lag_7 (toneladas de 7 meses atrás)
    8. tn_lag_8 (toneladas de 8 meses atrás)
    9. tn_lag_9 (t

In [9]:
# 📋 Preparar datos para entrenamiento - GRANULARIDAD POR PRODUCTO
print("PREPARACIÓN DE DATOS DE ENTRENAMIENTO POR PRODUCTO")
print("="*60)

# Filtrar registros con target válido
train_data = data_combined[data_combined['target'].notna()].copy()
print(f"📊 Registros válidos para entrenamiento: {len(train_data)}")

# Seleccionar features para el modelo con granularidad por producto
feature_columns = [
    # Lags de ventas (tn) por producto - EXPANDIDO A 36 LAGS (3 AÑOS)
    'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6',
    'tn_lag_7', 'tn_lag_8', 'tn_lag_9', 'tn_lag_10', 'tn_lag_11', 'tn_lag_12',
    'tn_lag_13', 'tn_lag_14', 'tn_lag_15', 'tn_lag_16', 'tn_lag_17', 'tn_lag_18',
    'tn_lag_19', 'tn_lag_20', 'tn_lag_21', 'tn_lag_22', 'tn_lag_23', 'tn_lag_24',
    'tn_lag_25', 'tn_lag_26', 'tn_lag_27', 'tn_lag_28', 'tn_lag_29', 'tn_lag_30',
    'tn_lag_31', 'tn_lag_32', 'tn_lag_33', 'tn_lag_34', 'tn_lag_35', 'tn_lag_36',
    
    # Lags de clientes por producto
    'num_customers_lag_1', 'num_customers_lag_2', 'num_customers_lag_3',
    
    # Lags de solicitudes por producto
    'total_request_tn_lag_1', 'total_request_tn_lag_2', 'total_request_tn_lag_3',
    
    # Lags de stock por producto
    'stock_tn_mean_lag_1', 'stock_tn_mean_lag_2', 'stock_tn_mean_lag_3',
    'stock_tn_sum_lag_1', 'stock_tn_sum_lag_2',
    
    # Features temporales
    'mes', 'trimestre', 'año',
    
    # Rolling features por producto
    'tn_rolling_mean_3', 'tn_rolling_mean_6',
    'tn_rolling_std_3', 'tn_rolling_std_6',
    
    # Features actuales por producto
    'num_customers', 'total_request_qty', 'total_request_tn',
    'stock_tn_mean', 'stock_tn_sum', 'stock_tn_std',
    
    # Feature de diversidad de clientes
    'cant_clientes_unicos',
    
    # Feature de longevidad del producto
    'meses_vida'
]

# Verificar qué features existen
available_features = [col for col in feature_columns if col in train_data.columns]
missing_features = [col for col in feature_columns if col not in train_data.columns]

print(f"✅ Features disponibles: {len(available_features)}")
print(f"⚠️ Features faltantes: {len(missing_features)}")
if missing_features:
    print(f"Features faltantes: {missing_features}")

# Usar solo features disponibles
feature_columns = available_features

# Preparar X e y
X = train_data[feature_columns].copy()
y = train_data['target'].copy()

# Rellenar valores nulos con 0 (para lags iniciales y stocks faltantes)
X = X.fillna(0)

print(f"📊 Shape de X: {X.shape}")
print(f"📈 Shape de y: {y.shape}")
print(f"🔍 Valores nulos en X: {X.isnull().sum().sum()}")
print(f"🔍 Valores nulos en y: {y.isnull().sum()}")

# # División temporal para validación (últimos períodos como validación)
# train_data_sorted = train_data.sort_values('fecha')
# split_date = train_data_sorted['fecha'].quantile(0.8)  # 80% entrenamiento, 20% validación

# train_mask = train_data_sorted['fecha'] <= split_date

# División temporal para validación usando fecha 201910
split_date = pd.to_datetime('201910', format='%Y%m')
train_mask = train_data['fecha'] < split_date
X_train = X.loc[train_mask]
X_val = X.loc[~train_mask] 
y_train = y.loc[train_mask]
y_val = y.loc[~train_mask]


print(f"\n📊 DIVISIÓN TEMPORAL:")
print(f"Entrenamiento: {len(X_train)} registros (hasta {split_date.strftime('%Y-%m')})")
print(f"Validación: {len(X_val)} registros (desde {split_date.strftime('%Y-%m')})")

print(f"\n🎯 PRODUCTOS EN ENTRENAMIENTO:")
productos_train = train_data.loc[train_mask, 'product_id'].nunique()
productos_val = train_data.loc[~train_mask, 'product_id'].nunique()
print(f"Productos únicos en entrenamiento: {productos_train}")
print(f"Productos únicos en validación: {productos_val}")

print(f"\nFeatures seleccionadas para granularidad por producto:")
for i, feat in enumerate(feature_columns):
    print(f"  {i+1:2d}. {feat}")

PREPARACIÓN DE DATOS DE ENTRENAMIENTO POR PRODUCTO
📊 Registros válidos para entrenamiento: 20789
✅ Features disponibles: 62
⚠️ Features faltantes: 0
📊 Shape de X: (20789, 62)
📈 Shape de y: (20789,)
🔍 Valores nulos en X: 0
🔍 Valores nulos en y: 0

📊 DIVISIÓN TEMPORAL:
Entrenamiento: 20009 registros (hasta 2019-10)
Validación: 780 registros (desde 2019-10)

🎯 PRODUCTOS EN ENTRENAMIENTO:
Productos únicos en entrenamiento: 780
Productos únicos en validación: 780

Features seleccionadas para granularidad por producto:
   1. tn_lag_1
   2. tn_lag_2
   3. tn_lag_3
   4. tn_lag_4
   5. tn_lag_5
   6. tn_lag_6
   7. tn_lag_7
   8. tn_lag_8
   9. tn_lag_9
  10. tn_lag_10
  11. tn_lag_11
  12. tn_lag_12
  13. tn_lag_13
  14. tn_lag_14
  15. tn_lag_15
  16. tn_lag_16
  17. tn_lag_17
  18. tn_lag_18
  19. tn_lag_19
  20. tn_lag_20
  21. tn_lag_21
  22. tn_lag_22
  23. tn_lag_23
  24. tn_lag_24
  25. tn_lag_25
  26. tn_lag_26
  27. tn_lag_27
  28. tn_lag_28
  29. tn_lag_29
  30. tn_lag_30
  31. tn_l

In [10]:
# 📊 Resumen detallado de features con lags de toneladas expandidos
print("RESUMEN DETALLADO DE FEATURES CON LAGS EXPANDIDOS")
print("="*60)

# Agrupar features por categoría
tn_lags = [f for f in feature_columns if f.startswith('tn_lag_')]
customer_lags = [f for f in feature_columns if f.startswith('num_customers_lag_')]
request_lags = [f for f in feature_columns if f.startswith('total_request_tn_lag_')]
stock_lags = [f for f in feature_columns if f.startswith('stock_tn_')]
temporal_features = [f for f in feature_columns if f in ['mes', 'trimestre', 'año']]
rolling_features = [f for f in feature_columns if 'rolling' in f]
current_features = [f for f in feature_columns if f in ['num_customers', 'total_request_qty', 'total_request_tn', 'stock_tn_mean', 'stock_tn_sum', 'stock_tn_std', 'cant_clientes_unicos', 'meses_vida']]

print(f"📈 LAGS DE TONELADAS (EXPANDIDOS A 36): {len(tn_lags)} features")
tn_lags_sorted = sorted(tn_lags, key=lambda x: int(x.split('_')[-1]))
for i, lag in enumerate(tn_lags_sorted, 1):
    lag_num = lag.split('_')[-1]
    print(f"   {i:2d}. {lag} (ventas de {lag_num} mes{'es' if int(lag_num) > 1 else ''} atrás)")

print(f"\n👥 LAGS DE CLIENTES: {len(customer_lags)} features")
for i, lag in enumerate(customer_lags, 1):
    print(f"   {i}. {lag}")

print(f"\n📋 LAGS DE SOLICITUDES: {len(request_lags)} features")
for i, lag in enumerate(request_lags, 1):
    print(f"   {i}. {lag}")

print(f"\n📦 FEATURES DE STOCK: {len(stock_lags)} features")
for i, lag in enumerate(stock_lags, 1):
    print(f"   {i}. {lag}")

print(f"\n📅 FEATURES TEMPORALES: {len(temporal_features)} features")
for i, feat in enumerate(temporal_features, 1):
    print(f"   {i}. {feat}")

print(f"\n📊 FEATURES ROLLING: {len(rolling_features)} features")
for i, feat in enumerate(rolling_features, 1):
    print(f"   {i}. {feat}")

print(f"\n🔄 FEATURES ACTUALES: {len(current_features)} features")
for i, feat in enumerate(current_features, 1):
    print(f"   {i}. {feat}")

print(f"\n✅ TOTAL DE FEATURES: {len(feature_columns)}")
print(f"🎯 MEJORA PRINCIPAL: Lags de toneladas expandidos de 12 → 36 features")
print(f"🔢 NUEVA FEATURE: cant_clientes_unicos (diversidad de base de clientes)")
print(f"🗓️ NUEVA FEATURE: meses_vida (longevidad del producto en el mercado)")
print(f"📈 Esto permite capturar:")
print(f"   • Patrones estacionales anuales completos (12 meses)")
print(f"   • Patrones estacionales bi-anuales (24 meses)")
print(f"   • Patrones estacionales tri-anuales (36 meses)")
print(f"   • Tendencias de corto plazo (1-6 meses)")
print(f"   • Tendencias de mediano plazo (7-18 meses)")
print(f"   • Tendencias de largo plazo (19-36 meses)")
print(f"   • Ciclos multi-anuales y efectos estacionales profundos")
print(f"   • Diversidad de base de clientes por producto")
print(f"   • Madurez y longevidad del producto en el mercado")

RESUMEN DETALLADO DE FEATURES CON LAGS EXPANDIDOS
📈 LAGS DE TONELADAS (EXPANDIDOS A 36): 36 features
    1. tn_lag_1 (ventas de 1 mes atrás)
    2. tn_lag_2 (ventas de 2 meses atrás)
    3. tn_lag_3 (ventas de 3 meses atrás)
    4. tn_lag_4 (ventas de 4 meses atrás)
    5. tn_lag_5 (ventas de 5 meses atrás)
    6. tn_lag_6 (ventas de 6 meses atrás)
    7. tn_lag_7 (ventas de 7 meses atrás)
    8. tn_lag_8 (ventas de 8 meses atrás)
    9. tn_lag_9 (ventas de 9 meses atrás)
   10. tn_lag_10 (ventas de 10 meses atrás)
   11. tn_lag_11 (ventas de 11 meses atrás)
   12. tn_lag_12 (ventas de 12 meses atrás)
   13. tn_lag_13 (ventas de 13 meses atrás)
   14. tn_lag_14 (ventas de 14 meses atrás)
   15. tn_lag_15 (ventas de 15 meses atrás)
   16. tn_lag_16 (ventas de 16 meses atrás)
   17. tn_lag_17 (ventas de 17 meses atrás)
   18. tn_lag_18 (ventas de 18 meses atrás)
   19. tn_lag_19 (ventas de 19 meses atrás)
   20. tn_lag_20 (ventas de 20 meses atrás)
   21. tn_lag_21 (ventas de 21 meses at

In [11]:
# 🔧 Optimización de hiperparámetros con Optuna para LINEAR_TREE
print("OPTIMIZACIÓN DE HIPERPARÁMETROS CON OPTUNA - LINEAR_TREE")
print("="*60)

# 🎲 Configurar semilla para Optuna y reproductibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

def objective(trial):
    """Función objetivo para optimización con Optuna incluyendo linear_tree"""
    
    # 🎲 Configurar semilla para cada trial
    np.random.seed(SEED + trial.number)
    random.seed(SEED + trial.number)
    
    # Sugerir hiperparámetros optimizados para linear_tree
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'linear_tree': True,        # FIJO: Característica principal del modelo
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 2.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 2.0),
        'num_leaves': trial.suggest_int('num_leaves', 20, 120),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 40),
        'max_bin': trial.suggest_int('max_bin', 100, 400),
        'verbose': -1,
        'random_state': SEED + trial.number,  # Semilla única por trial
        'bagging_seed': SEED + trial.number,  # Semilla para bagging
        'feature_fraction_seed': SEED + trial.number  # Semilla para feature selection
    }
    
    # Crear datasets de LightGBM
    train_dataset = lgb.Dataset(X_train, label=y_train)
    val_dataset = lgb.Dataset(X_val, label=y_val, reference=train_dataset)
    
    # Entrenar modelo con validación cruzada interna
    model = lgb.train(
        params,
        train_dataset,
        valid_sets=[val_dataset],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(0)  # Silenciar logs
        ]
    )
    
    # Predecir en conjunto de validación
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    
    # Calcular MAE como métrica a minimizar
    mae = mean_absolute_error(y_val, y_pred)
    
    return mae

# Crear estudio de optimización
print("🔍 Iniciando optimización de hiperparámetros con LINEAR_TREE...")
print(f"🎲 Usando semilla base {SEED} para reproducibilidad completa")

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=SEED),  # Semilla para el sampler
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

# Ejecutar optimización
n_trials = 700 # 500 Número de pruebas
print(f"🚀 Ejecutando {n_trials} trials de optimización para LINEAR_TREE...")
print(f"🎲 Cada trial usa semilla: SEED + trial_number para reproductibilidad")

study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

# Mostrar mejores parámetros
print(f"\n✅ Optimización completada!")
print(f"🏆 Mejor MAE encontrado: {study.best_value:.4f}")
print(f"\n🔧 HIPERPARÁMETROS OPTIMIZADOS PARA LINEAR_TREE:")
best_params = study.best_params
print("="*65)
print(f"   linear_tree:      True (FIJO - Característica principal) ⭐")
print(f"   lambda_l1:        {best_params['lambda_l1']:.4f}")
print(f"   lambda_l2:        {best_params['lambda_l2']:.4f}")
print(f"   num_leaves:       {best_params['num_leaves']}")
print(f"   feature_fraction: {best_params['feature_fraction']:.4f}")
print(f"   learning_rate:    {best_params['learning_rate']:.4f}")
print(f"   bagging_fraction: {best_params['bagging_fraction']:.4f}")
print(f"   bagging_freq:     {best_params['bagging_freq']}")
print(f"   min_child_samples: {best_params['min_child_samples']}")
print(f"   max_bin:          {best_params['max_bin']}")
print("="*65)

# Agregar parámetros fijos
best_params.update({
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'linear_tree': True,  # Mantener como característica principal
    'verbose': 0,
    'random_state': SEED,  # Semilla principal para el modelo final
    'bagging_seed': SEED,  # Semilla para bagging
    'feature_fraction_seed': SEED,  # Semilla para feature selection
    'data_seed': SEED  # Semilla para datos
})

print(f"\n📊 Resumen de la optimización:")
print(f"  Trials completados: {len(study.trials)}")
print(f"  Mejor trial: {study.best_trial.number}")
print(f"  Tiempo total: {sum(t.duration.total_seconds() for t in study.trials if t.duration):.1f} segundos")
print(f"🎲 Reproductibilidad garantizada con SEED={SEED}")

[I 2025-08-10 10:32:37,370] A new study created in memory with name: no-name-acb13700-ca13-44df-ae0e-955c17bc0d9e


OPTIMIZACIÓN DE HIPERPARÁMETROS CON OPTUNA - LINEAR_TREE
🔍 Iniciando optimización de hiperparámetros con LINEAR_TREE...
🎲 Usando semilla base 42 para reproducibilidad completa
🚀 Ejecutando 700 trials de optimización para LINEAR_TREE...
🎲 Cada trial usa semilla: SEED + trial_number para reproductibilidad


  0%|          | 0/700 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[184]	valid_0's l1: 10.52
[I 2025-08-10 10:32:39,111] Trial 0 finished with value: 10.51996470092889 and parameters: {'lambda_l1': 0.749080237694725, 'lambda_l2': 1.9014286128198323, 'num_leaves': 93, 'feature_fraction': 0.8394633936788146, 'learning_rate': 0.015257808482051183, 'bagging_fraction': 0.662397808134481, 'bagging_freq': 1, 'min_child_samples': 36, 'max_bin': 280}. Best is trial 0 with value: 10.51996470092889.
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[130]	valid_0's l1: 13.2161
[I 2025-08-10 10:32:40,844] Trial 1 finished with value: 13.216124305573011 and parameters: {'lambda_l1': 1.416145155592091, 'lambda_l2': 0.041168988591604894, 'num_leaves': 117, 'feature_fraction': 0.9329770563201687, 'learning_rate': 0.01777174904859463, 'bagging_fraction': 0.6727299868828402, 'bagging_freq': 2, 'min_child_samples': 15, 'max_bin': 2

In [12]:
# 🚀 Entrenar 20 modelos LightGBM con diferentes semillas - LINEAR_TREE
print("ENTRENAMIENTO DE 20 MODELOS LIGHTGBM CON DIFERENTES SEMILLAS - LINEAR_TREE")
print("="*80)

# 🎲 Definir las 20 semillas diferentes
semillas = [42, 109, 113, 151, 167, 179, 193, 199, 211, 241,
            263, 281, 307, 331, 367, 401, 439, 487, 563, 617]

print(f"🔢 Usando {len(semillas)} semillas diferentes para entrenamiento:")
print(f"   Semillas: {semillas}")

# Usar los mejores parámetros encontrados por Optuna
lgb_params_base = best_params.copy()

# Listas para almacenar modelos y métricas
modelos = []
metricas = []
predicciones_val = []
predicciones_finales = []

# 💾 Crear directorio para guardar los modelos
import os
import pickle
modelos_dir = 'data/modelos_entrenados/sem_1_e3'
os.makedirs(modelos_dir, exist_ok=True)

print(f"\n💾 Los modelos se guardarán en: {modelos_dir}/")
print(f"\n🚀 Iniciando entrenamiento de {len(semillas)} modelos...")

for i, seed in enumerate(semillas, 1):
    print(f"\n{'='*60}")
    print(f"🎯 ENTRENANDO MODELO {i}/20 - SEMILLA: {seed}")
    print(f"{'='*60}")
    
    # 🎲 Configurar semilla específica para este modelo
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear parámetros específicos para este modelo
    lgb_params = lgb_params_base.copy()
    lgb_params.update({
        'random_state': seed,
        'bagging_seed': seed,
        'feature_fraction_seed': seed,
        'data_seed': seed
    })
    
    print(f"🔧 Parámetros para modelo {i}:")
    print(f"   linear_tree: {lgb_params['linear_tree']} ⭐")
    print(f"   random_state: {seed}")
    print(f"   learning_rate: {lgb_params['learning_rate']:.4f}")
    print(f"   num_leaves: {lgb_params['num_leaves']}")
    
    # Crear datasets de LightGBM
    train_dataset = lgb.Dataset(X_train, label=y_train)
    val_dataset = lgb.Dataset(X_val, label=y_val, reference=train_dataset)
    
    # Entrenar el modelo
    modelo = lgb.train(
        lgb_params,
        train_dataset,
        valid_sets=[train_dataset, val_dataset],
        valid_names=['train', 'eval'],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=150),
            lgb.log_evaluation(0)  # Silenciar logs individuales
        ]
    )
    
    # 💾 Guardar el modelo individual con su semilla
    model_filename = f"{modelos_dir}/modelo_lgbm_seed_{seed}.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(modelo, f)
    print(f"💾 Modelo guardado en: {model_filename}")
    
    # Predicciones en validación
    y_pred_val = modelo.predict(X_val, num_iteration=modelo.best_iteration)
    
    # Calcular métricas
    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100
    
    # Almacenar resultados (incluyendo ruta del modelo)
    modelos.append(modelo)
    metricas.append({
        'modelo': i,
        'semilla': seed,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'num_trees': modelo.num_trees(),
        'model_file': model_filename  # 📁 Ruta del archivo del modelo
    })
    predicciones_val.append(y_pred_val)
    
    print(f"✅ Modelo {i} entrenado exitosamente!")
    print(f"   MAE:  {mae:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   MAPE: {mape:.2f}%")
    print(f"   Árboles: {modelo.num_trees()}")

print(f"\n🎉 ¡TODOS LOS {len(semillas)} MODELOS ENTRENADOS EXITOSAMENTE!")
print(f"⭐ Cada modelo usa árboles lineales híbridos con parámetros optimizados por Optuna")
print(f"💾 Todos los modelos han sido guardados en: {modelos_dir}/")

# Crear DataFrame con métricas (incluyendo rutas de archivos)
df_metricas = pd.DataFrame(metricas)
print(f"\n📊 RESUMEN DE MÉTRICAS DE LOS {len(semillas)} MODELOS:")
print(f"{'='*60}")
print(df_metricas[['modelo', 'semilla', 'mae', 'rmse', 'mape', 'num_trees']])  # Mostrar sin la ruta por claridad

print(f"\n📈 ESTADÍSTICAS DE LAS MÉTRICAS:")
print(f"   MAE  - Promedio: {df_metricas['mae'].mean():.4f}, Std: {df_metricas['mae'].std():.4f}")
print(f"   RMSE - Promedio: {df_metricas['rmse'].mean():.4f}, Std: {df_metricas['rmse'].std():.4f}")
print(f"   MAPE - Promedio: {df_metricas['mape'].mean():.2f}%, Std: {df_metricas['mape'].std():.2f}%")

# Encontrar el mejor modelo
mejor_modelo_idx = df_metricas['mae'].idxmin()
mejor_modelo = modelos[mejor_modelo_idx]
mejor_semilla = semillas[mejor_modelo_idx]
mejor_model_file = df_metricas.loc[mejor_modelo_idx, 'model_file']

print(f"\n🏆 MEJOR MODELO:")
print(f"   Modelo: {mejor_modelo_idx + 1}")
print(f"   Semilla: {mejor_semilla}")
print(f"   MAE: {df_metricas.loc[mejor_modelo_idx, 'mae']:.4f}")
print(f"   RMSE: {df_metricas.loc[mejor_modelo_idx, 'rmse']:.4f}")
print(f"   MAPE: {df_metricas.loc[mejor_modelo_idx, 'mape']:.2f}%")
print(f"   📁 Archivo: {mejor_model_file}")

print(f"\n📁 LISTA DE MODELOS GUARDADOS:")
for i, row in df_metricas.iterrows():
    print(f"   Modelo {row['modelo']:2d} (seed {row['semilla']:3d}): {row['model_file']}")

ENTRENAMIENTO DE 20 MODELOS LIGHTGBM CON DIFERENTES SEMILLAS - LINEAR_TREE
🔢 Usando 20 semillas diferentes para entrenamiento:
   Semillas: [42, 109, 113, 151, 167, 179, 193, 199, 211, 241, 263, 281, 307, 331, 367, 401, 439, 487, 563, 617]

💾 Los modelos se guardarán en: data/modelos_entrenados/sem_1_e3/

🚀 Iniciando entrenamiento de 20 modelos...

🎯 ENTRENANDO MODELO 1/20 - SEMILLA: 42
🔧 Parámetros para modelo 1:
   linear_tree: True ⭐
   random_state: 42
   learning_rate: 0.1132
   num_leaves: 35
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[45]	train's l1: 8.25387	eval's l1: 8.81637
💾 Modelo guardado en: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_42.pkl
✅ Modelo 1 entrenado exitosamente!
   MAE:  8.8164
   RMSE: 22.4672
   MAPE: 290.08%
   Árboles: 45

🎯 ENTRENANDO MODELO 2/20 - SEMILLA: 109
🔧 Parámetros para modelo 2:
   linear_tree: True ⭐
   random_state: 109
   learning_rate: 0.1132
   num_leaves: 35
Training until valid

In [13]:
# # 📊 Análisis de importancia de features
# print("ANÁLISIS DE IMPORTANCIA DE FEATURES")
# print("="*50)

# # Obtener importancia de features
# feature_importance = model.feature_importance(importance_type='gain')
# feature_names = feature_columns

# # Crear DataFrame con importancias
# importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': feature_importance
# }).sort_values('importance', ascending=False)

# print("🔝 Top 10 features más importantes:")
# print(importance_df.head(10))

# # Visualizar importancia
# plt.figure(figsize=(10, 8))
# top_features = importance_df.head(15)
# plt.barh(range(len(top_features)), top_features['importance'])
# plt.yticks(range(len(top_features)), top_features['feature'])
# plt.xlabel('Importancia')
# plt.title('Top 15 Features - Importancia LightGBM')
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

# # Análisis de predicciones
# print(f"\n🎯 ANÁLISIS DE PREDICCIONES:")
# print(f"Predicciones mínimas: {y_pred_val.min():.2f}")
# print(f"Predicciones máximas: {y_pred_val.max():.2f}")
# print(f"Predicciones promedio: {y_pred_val.mean():.2f}")
# print(f"Valores reales promedio: {y_val.mean():.2f}")

# # Verificar predicciones negativas
# negative_preds = (y_pred_val < 0).sum()
# print(f"Predicciones negativas: {negative_preds} ({negative_preds/len(y_pred_val)*100:.1f}%)")

In [14]:
# 🔧 Funciones de utilidad para cargar modelos guardados
import pickle
import os

def cargar_modelo_por_semilla(semilla, modelos_dir='data/modelos_entrenados/sem_1_e3'):
    """
    Cargar un modelo específico por su semilla
    
    Args:
        semilla (int): La semilla del modelo a cargar
        modelos_dir (str): Directorio donde están guardados los modelos
    
    Returns:
        lightgbm.Booster: El modelo cargado
    """
    model_path = f"{modelos_dir}/modelo_lgbm_seed_{semilla}.pkl"
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            modelo = pickle.load(f)
        print(f"✅ Modelo con semilla {semilla} cargado desde: {model_path}")
        return modelo
    else:
        raise FileNotFoundError(f"❌ No se encontró el modelo con semilla {semilla} en {model_path}")

def cargar_todos_los_modelos(semillas, modelos_dir='data/modelos_entrenados/sem_1_e3'):
    """
    Cargar todos los modelos por sus semillas
    
    Args:
        semillas (list): Lista de semillas de los modelos a cargar
        modelos_dir (str): Directorio donde están guardados los modelos
    
    Returns:
        list: Lista de modelos cargados
    """
    modelos_cargados = []
    print(f"🔄 Cargando {len(semillas)} modelos desde {modelos_dir}/...")
    
    for i, semilla in enumerate(semillas, 1):
        try:
            modelo = cargar_modelo_por_semilla(semilla, modelos_dir)
            modelos_cargados.append(modelo)
            print(f"   {i:2d}/20 - Modelo semilla {semilla} ✅")
        except FileNotFoundError as e:
            print(f"   {i:2d}/20 - Error: {e}")
            return None
    
    print(f"🎉 Todos los {len(modelos_cargados)} modelos cargados exitosamente!")
    return modelos_cargados

def cargar_mejor_modelo(df_metricas, modelos_dir='data/modelos_entrenados/sem_1_e3'):
    """
    Cargar el mejor modelo basado en las métricas
    
    Args:
        df_metricas (pd.DataFrame): DataFrame con las métricas de los modelos
        modelos_dir (str): Directorio donde están guardados los modelos
    
    Returns:
        tuple: (modelo, semilla, mae)
    """
    mejor_idx = df_metricas['mae'].idxmin()
    mejor_semilla = df_metricas.loc[mejor_idx, 'semilla']
    mejor_mae = df_metricas.loc[mejor_idx, 'mae']
    
    mejor_modelo = cargar_modelo_por_semilla(mejor_semilla, modelos_dir)
    print(f"🏆 Mejor modelo cargado: Semilla {mejor_semilla} (MAE: {mejor_mae:.4f})")
    
    return mejor_modelo, mejor_semilla, mejor_mae

def generar_predicciones_ensemble(modelos_cargados, X_pred, estrategia='promedio'):
    """
    Generar predicciones usando ensemble de modelos
    
    Args:
        modelos_cargados (list): Lista de modelos cargados
        X_pred (pd.DataFrame): Features para predecir
        estrategia (str): 'promedio', 'mediana', o 'weighted'
    
    Returns:
        np.array: Predicciones del ensemble
    """
    print(f"🔮 Generando predicciones con ensemble de {len(modelos_cargados)} modelos...")
    
    todas_predicciones = []
    for i, modelo in enumerate(modelos_cargados, 1):
        predicciones = modelo.predict(X_pred, num_iteration=modelo.best_iteration)
        predicciones = np.maximum(predicciones, 0)  # Eliminar predicciones negativas
        todas_predicciones.append(predicciones)
        print(f"   Predicciones modelo {i}/20 generadas ✅")
    
    todas_predicciones = np.array(todas_predicciones)
    
    if estrategia == 'promedio':
        resultado = np.mean(todas_predicciones, axis=0)
        print(f"📊 Ensemble promedio calculado")
    elif estrategia == 'mediana':
        resultado = np.median(todas_predicciones, axis=0)
        print(f"📊 Ensemble mediana calculado")
    else:
        # Por defecto usar promedio
        resultado = np.mean(todas_predicciones, axis=0)
        print(f"📊 Ensemble promedio calculado (estrategia por defecto)")
    
    return resultado, todas_predicciones

print("🔧 Funciones de utilidad para manejo de modelos definidas:")
print("   - cargar_modelo_por_semilla(semilla)")
print("   - cargar_todos_los_modelos(semillas)")
print("   - cargar_mejor_modelo(df_metricas)")
print("   - generar_predicciones_ensemble(modelos, X_pred, estrategia)")
print("✅ Ahora puedes cargar y usar los modelos individuales para ensembles")

🔧 Funciones de utilidad para manejo de modelos definidas:
   - cargar_modelo_por_semilla(semilla)
   - cargar_todos_los_modelos(semillas)
   - cargar_mejor_modelo(df_metricas)
   - generar_predicciones_ensemble(modelos, X_pred, estrategia)
✅ Ahora puedes cargar y usar los modelos individuales para ensembles


In [15]:
# 🔮 Generar predicciones finales cargando los modelos guardados
print("GENERACIÓN DE PREDICCIONES FINALES CARGANDO MODELOS GUARDADOS")
print("="*70)

# Preparar datos para predicción (últimos datos disponibles de cada producto)
ultimo_periodo = data_combined.groupby('product_id')['fecha'].max().reset_index()
ultimo_periodo.columns = ['product_id', 'ultima_fecha']

# Unir con datos completos para obtener features más recientes
datos_prediccion = pd.merge(data_combined, ultimo_periodo, on='product_id')
datos_prediccion = datos_prediccion[datos_prediccion['fecha'] == datos_prediccion['ultima_fecha']].copy()

print(f"📊 Productos para predicción: {len(datos_prediccion)}")
print(f"📅 Período base para predicción: {datos_prediccion['periodo'].value_counts().head()}")

# Preparar features para predicción
X_pred = datos_prediccion[feature_columns].copy()
X_pred = X_pred.fillna(0)

print(f"🔍 Shape de datos de predicción: {X_pred.shape}")
print(f"🔍 Valores nulos en predicción: {X_pred.isnull().sum().sum()}")

# 💾 OPCIÓN 1: Cargar todos los modelos desde disco para ensemble completo
print(f"\n💾 CARGANDO MODELOS DESDE DISCO PARA ENSEMBLE...")
modelos_cargados = cargar_todos_los_modelos(semillas)

if modelos_cargados is not None:
    # Generar predicciones con ensemble
    predicciones_promedio, todas_las_predicciones = generar_predicciones_ensemble(
        modelos_cargados, X_pred, estrategia='promedio'
    )
    
    predicciones_mediana, _ = generar_predicciones_ensemble(
        modelos_cargados, X_pred, estrategia='mediana'
    )
    
    print(f"✅ Ensemble completo generado con {len(modelos_cargados)} modelos")
else:
    # Fallback: usar modelos en memoria si fallan las cargas
    print("⚠️ Fallback: Usando modelos en memoria...")
    todas_las_predicciones = []
    for i, modelo in enumerate(modelos, 1):
        print(f"   Generando predicciones con modelo {i} (semilla {semillas[i-1]})...")
        predicciones = modelo.predict(X_pred, num_iteration=modelo.best_iteration)
        predicciones = np.maximum(predicciones, 0)
        todas_las_predicciones.append(predicciones)
    
    todas_las_predicciones = np.array(todas_las_predicciones)
    predicciones_promedio = np.mean(todas_las_predicciones, axis=0)
    predicciones_mediana = np.median(todas_las_predicciones, axis=0)

# 🏆 OPCIÓN 2: Cargar solo el mejor modelo
print(f"\n🏆 CARGANDO MEJOR MODELO INDIVIDUAL...")
mejor_modelo_cargado, mejor_semilla_cargada, mejor_mae_cargado = cargar_mejor_modelo(df_metricas)
mejor_predicciones = mejor_modelo_cargado.predict(X_pred, num_iteration=mejor_modelo_cargado.best_iteration)
mejor_predicciones = np.maximum(mejor_predicciones, 0)

# Calcular estadísticas adicionales
predicciones_std = np.std(todas_las_predicciones, axis=0)
predicciones_min = np.min(todas_las_predicciones, axis=0)
predicciones_max = np.max(todas_las_predicciones, axis=0)

print(f"\n📊 ESTADÍSTICAS DE LAS PREDICCIONES:")
print(f"   Promedio de ensemble promedio: {predicciones_promedio.mean():.2f}")
print(f"   Promedio de ensemble mediana: {predicciones_mediana.mean():.2f}")
print(f"   Promedio de desviación estándar: {predicciones_std.mean():.2f}")
print(f"   Promedio del mejor modelo: {mejor_predicciones.mean():.2f}")

# Crear DataFrames de resultados
print(f"\n💾 Creando DataFrames de resultados...")

# DataFrame con todas las predicciones individuales
df_predicciones_individuales = pd.DataFrame({
    'product_id': datos_prediccion['product_id'].values
})

for i, semilla in enumerate(semillas):
    df_predicciones_individuales[f'pred_modelo_{i+1}_seed_{semilla}'] = todas_las_predicciones[i]

# DataFrame con estadísticas agregadas
resultado_lgbm_estadisticas = pd.DataFrame({
    'product_id': datos_prediccion['product_id'].values,
    'tn_promedio': predicciones_promedio,
    'tn_mediana': predicciones_mediana,
    'tn_std': predicciones_std,
    'tn_min': predicciones_min,
    'tn_max': predicciones_max,
    'tn_mejor_modelo': mejor_predicciones
})

# DataFrame principal (para compatibilidad) - usando promedio
resultado_lgbm = pd.DataFrame({
    'product_id': datos_prediccion['product_id'].values,
    'tn': predicciones_promedio
})

print(f"✅ Predicciones generadas para {len(resultado_lgbm)} productos")
print(f"\n📊 Estadísticas del ensemble promedio:")
print(f"  Promedio: {resultado_lgbm['tn'].mean():.2f}")
print(f"  Mediana:  {resultado_lgbm['tn'].median():.2f}")
print(f"  Mínimo:   {resultado_lgbm['tn'].min():.2f}")
print(f"  Máximo:   {resultado_lgbm['tn'].max():.2f}")
print(f"  Std:      {resultado_lgbm['tn'].std():.2f}")

print(f"\n📊 Estadísticas del mejor modelo:")
print(f"  Promedio: {mejor_predicciones.mean():.2f}")
print(f"  Mediana:  {np.median(mejor_predicciones):.2f}")
print(f"  Mínimo:   {mejor_predicciones.min():.2f}")
print(f"  Máximo:   {mejor_predicciones.max():.2f}")
print(f"  Std:      {mejor_predicciones.std():.2f}")

print(f"\n💾 DEMOSTRACIÓN DE FUNCIONALIDAD:")
print(f"✅ Modelos guardados individualmente por semilla")
print(f"✅ Ensemble promedio generado cargando modelos desde disco")
print(f"✅ Mejor modelo individual cargado por semilla {mejor_semilla_cargada}")
print(f"✅ Funciones disponibles para cargar cualquier modelo específico")

print(f"\nPrimeras 10 predicciones (ensemble promedio):")
print(resultado_lgbm.head(10))

GENERACIÓN DE PREDICCIONES FINALES CARGANDO MODELOS GUARDADOS
📊 Productos para predicción: 780
📅 Período base para predicción: periodo
201912    780
Name: count, dtype: int64
🔍 Shape de datos de predicción: (780, 62)
🔍 Valores nulos en predicción: 0

💾 CARGANDO MODELOS DESDE DISCO PARA ENSEMBLE...
🔄 Cargando 20 modelos desde data/modelos_entrenados/sem_1_e3/...
✅ Modelo con semilla 42 cargado desde: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_42.pkl
    1/20 - Modelo semilla 42 ✅
✅ Modelo con semilla 109 cargado desde: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_109.pkl
    2/20 - Modelo semilla 109 ✅
✅ Modelo con semilla 113 cargado desde: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_113.pkl
    3/20 - Modelo semilla 113 ✅
✅ Modelo con semilla 151 cargado desde: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_151.pkl
    4/20 - Modelo semilla 151 ✅
✅ Modelo con semilla 167 cargado desde: data/modelos_entrenados/sem_1_e3/modelo_lgbm_seed_167.pkl
    5/20 - Modelo sem

In [ ]:
# 💾 Guardar todos los archivos de predicciones y información de modelos
import os
os.makedirs('data', exist_ok=True)

print("GUARDANDO ARCHIVOS DE PREDICCIONES Y MODELOS")
print("="*60)

# 1. Guardar predicciones promedio (archivo principal)
archivo_promedio = 'data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_promedio_e3.csv'
resultado_lgbm.to_csv(archivo_promedio, index=False)
print(f"✅ Predicciones promedio guardadas en: {archivo_promedio}")

# 2. Guardar predicciones del mejor modelo
archivo_mejor = 'data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_mejor_e3.csv'
resultado_mejor = pd.DataFrame({
    'product_id': datos_prediccion['product_id'].values,
    'tn': mejor_predicciones
})
resultado_mejor.to_csv(archivo_mejor, index=False)
print(f"✅ Predicciones del mejor modelo guardadas en: {archivo_mejor}")

# 3. Guardar estadísticas completas
archivo_estadisticas = 'data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_estadisticas_e3.csv'
resultado_lgbm_estadisticas.to_csv(archivo_estadisticas, index=False)
print(f"✅ Estadísticas completas guardadas en: {archivo_estadisticas}")

# 4. Guardar todas las predicciones individuales
archivo_individuales = 'data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_individuales_e3.csv'
df_predicciones_individuales.to_csv(archivo_individuales, index=False)
print(f"✅ Predicciones individuales guardadas en: {archivo_individuales}")

# 5. Guardar métricas de los modelos (incluyendo rutas de archivos)
archivo_metricas = 'data/metricas_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e3.csv'
df_metricas.to_csv(archivo_metricas, index=False)
print(f"✅ Métricas de los modelos guardadas en: {archivo_metricas}")

# 6. Guardar inventario de modelos guardados
archivo_inventario = 'data/inventario_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e3.csv'
inventario_modelos = df_metricas[['modelo', 'semilla', 'mae', 'rmse', 'mape', 'model_file']].copy()
inventario_modelos['modelo_existe'] = inventario_modelos['model_file'].apply(os.path.exists)
inventario_modelos.to_csv(archivo_inventario, index=False)
print(f"✅ Inventario de modelos guardado en: {archivo_inventario}")

print(f"\n📁 RESUMEN DE ARCHIVOS GUARDADOS:")
print(f"   1. {archivo_promedio} - Predicciones promedio de los 20 modelos")
print(f"   2. {archivo_mejor} - Predicciones del mejor modelo (semilla {mejor_semilla})")
print(f"   3. {archivo_estadisticas} - Estadísticas completas (promedio, mediana, std, min, max)")
print(f"   4. {archivo_individuales} - Predicciones de cada uno de los 20 modelos")
print(f"   5. {archivo_metricas} - Métricas de rendimiento de cada modelo")
print(f"   6. {archivo_inventario} - Inventario de modelos guardados con ubicaciones")

print(f"\n🤖 MODELOS INDIVIDUALES GUARDADOS:")
print(f"   Directorio: {modelos_dir}/")
for i, row in df_metricas.iterrows():
    existe = "✅" if os.path.exists(row['model_file']) else "❌"
    print(f"   {existe} modelo_lgbm_seed_{row['semilla']}.pkl (MAE: {row['mae']:.4f})")

print(f"\n🎯 INSTRUCCIONES DE USO:")
print(f"   📖 Para cargar un modelo específico:")
print(f"      modelo = cargar_modelo_por_semilla(semilla)")
print(f"   📖 Para cargar el mejor modelo:")
print(f"      mejor_modelo, semilla, mae = cargar_mejor_modelo(df_metricas)")
print(f"   📖 Para generar ensemble personalizado:")
print(f"      semillas_subset = [42, 113, 241]  # Ejemplo")
print(f"      modelos_subset = cargar_todos_los_modelos(semillas_subset)")
print(f"      predicciones = generar_predicciones_ensemble(modelos_subset, X_pred)")

print(f"\n✅ SISTEMA DE MODELOS PERSISTENTES IMPLEMENTADO EXITOSAMENTE!")
print(f"🔄 Los modelos están disponibles para reutilización y experimentos futuros")

GUARDANDO ARCHIVOS DE PREDICCIONES Y MODELOS
✅ Predicciones promedio guardadas en: data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_promedio_e2.csv
✅ Predicciones del mejor modelo guardadas en: data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_mejor_e2.csv
✅ Estadísticas completas guardadas en: data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_estadisticas_e2.csv
✅ Predicciones individuales guardadas en: data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_individuales_e2.csv
✅ Métricas de los modelos guardadas en: data/metricas_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e2.csv
✅ Inventario de modelos guardado en: data/inventario_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e2.csv

📁 RESUMEN DE ARCHIVOS GUARDADOS:
   1. data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_promedio_e2.csv - Predicciones promedio de los 20 modelos
   2. data/pred_lgbm_v3_FE_02_v3_linear_tree_opt

In [17]:
print(f"🎉 ¡SISTEMA DE ENSEMBLE CON MODELOS PERSISTENTES COMPLETADO!")
print(f"⭐ Cada modelo usa árboles lineales híbridos con optimización Optuna")
print(f"🔢 20 modelos entrenados con semillas diferentes para mayor robustez")
print(f"💾 Todos los modelos guardados individualmente para reutilización")
print(f"🏆 Mejor modelo individual: Semilla {mejor_semilla} (MAE: {df_metricas.loc[mejor_modelo_idx, 'mae']:.4f})")
print(f"📊 MAE promedio del conjunto: {df_metricas['mae'].mean():.4f} (±{df_metricas['mae'].std():.4f})")
print(f"📁 6 archivos de predicciones + 20 modelos individuales guardados")
print(f"🔧 Funciones de utilidad disponibles para cargar modelos por semilla")
print(f"🎲 Semillas utilizadas: {semillas}")
print(f"🏆 Tecnología: LINEAR_TREE + OPTUNA + 36 LAGS + ENSEMBLE PERSISTENTE DE 20 MODELOS")
print(f"")
print(f"🎯 NUEVAS CAPACIDADES DISPONIBLES:")
print(f"   🔄 Cargar cualquier modelo por su semilla")
print(f"   🏆 Cargar automáticamente el mejor modelo")
print(f"   📊 Generar ensembles personalizados con subconjuntos de modelos")
print(f"   💾 Reutilizar modelos sin re-entrenar")
print(f"   🔬 Experimentar con diferentes estrategias de ensemble")

🎉 ¡SISTEMA DE ENSEMBLE CON MODELOS PERSISTENTES COMPLETADO!
⭐ Cada modelo usa árboles lineales híbridos con optimización Optuna
🔢 20 modelos entrenados con semillas diferentes para mayor robustez
💾 Todos los modelos guardados individualmente para reutilización
🏆 Mejor modelo individual: Semilla 113 (MAE: 8.5922)
📊 MAE promedio del conjunto: 9.1058 (±0.4433)
📁 6 archivos de predicciones + 20 modelos individuales guardados
🔧 Funciones de utilidad disponibles para cargar modelos por semilla
🎲 Semillas utilizadas: [42, 109, 113, 151, 167, 179, 193, 199, 211, 241, 263, 281, 307, 331, 367, 401, 439, 487, 563, 617]
🏆 Tecnología: LINEAR_TREE + OPTUNA + 36 LAGS + ENSEMBLE PERSISTENTE DE 20 MODELOS

🎯 NUEVAS CAPACIDADES DISPONIBLES:
   🔄 Cargar cualquier modelo por su semilla
   🏆 Cargar automáticamente el mejor modelo
   📊 Generar ensembles personalizados con subconjuntos de modelos
   💾 Reutilizar modelos sin re-entrenar
   🔬 Experimentar con diferentes estrategias de ensemble


In [ ]:
# Guardar información de reproductibilidad del conjunto de modelos con persistencia
import json
os.makedirs('data', exist_ok=True)

# Crear información detallada de reproductibilidad
reproducibility_info_ensemble = {
    'experimento': 'LGBM_v3_FE_02_v3_linear_tree_opt_03_36lags_semillerio_persistente',
    'descripcion': 'Conjunto de 20 modelos LightGBM con linear_tree, parámetros optimizados por Optuna y persistencia individual',
    'numero_modelos': len(semillas),
    'semillas_utilizadas': semillas,
    'semilla_optuna': 42,
    'modelos_guardados': {
        'directorio': modelos_dir,
        'formato': 'pickle (.pkl)',
        'patron_nombre': 'modelo_lgbm_seed_{semilla}.pkl',
        'total_archivos': len(semillas)
    },
    'mejor_modelo': {
        'indice': int(mejor_modelo_idx + 1),
        'semilla': int(mejor_semilla),
        'mae': float(df_metricas.loc[mejor_modelo_idx, 'mae']),
        'rmse': float(df_metricas.loc[mejor_modelo_idx, 'rmse']),
        'mape': float(df_metricas.loc[mejor_modelo_idx, 'mape']),
        'archivo': mejor_model_file
    },
    'estadisticas_conjunto': {
        'mae_promedio': float(df_metricas['mae'].mean()),
        'mae_std': float(df_metricas['mae'].std()),
        'rmse_promedio': float(df_metricas['rmse'].mean()),
        'rmse_std': float(df_metricas['rmse'].std()),
        'mape_promedio': float(df_metricas['mape'].mean()),
        'mape_std': float(df_metricas['mape'].std())
    },
    'parametros_optimizados': {
        'linear_tree': lgb_params_base['linear_tree'],
        'lambda_l1': lgb_params_base['lambda_l1'],
        'lambda_l2': lgb_params_base['lambda_l2'],
        'num_leaves': lgb_params_base['num_leaves'],
        'feature_fraction': lgb_params_base['feature_fraction'],
        'learning_rate': lgb_params_base['learning_rate'],
        'bagging_fraction': lgb_params_base['bagging_fraction'],
        'bagging_freq': lgb_params_base['bagging_freq'],
        'min_child_samples': lgb_params_base['min_child_samples'],
        'max_bin': lgb_params_base['max_bin']
    },
    'configuracion_entrenamiento': {
        'num_boost_round': 2000,
        'early_stopping_rounds': 150,
        'validation_split': 0.8
    },
    'features_utilizadas': len(feature_columns),
    'lags_toneladas': 36,
    'funciones_utilidad': [
        'cargar_modelo_por_semilla(semilla)',
        'cargar_todos_los_modelos(semillas)',
        'cargar_mejor_modelo(df_metricas)',
        'generar_predicciones_ensemble(modelos, X_pred, estrategia)'
    ],
    'archivos_generados': [
        'pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_promedio_e3.csv',
        'pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_mejor_e3.csv',
        'pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_estadisticas_e3.csv',
        'pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_individuales_e3.csv',
        'metricas_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e3.csv',
        'inventario_modelos_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_sem_1_e3.csv'
    ],
    'modelos_individuales': [f"modelo_lgbm_seed_{seed}.pkl" for seed in semillas]
}

# Guardar archivo de reproductibilidad
archivo_reproducibilidad = 'data/reproducibility_info_lgbm_v3_FE_02_v3_linear_tree_opt_36lags_sem_1_ensemble_persistente_e3.json'
with open(archivo_reproducibilidad, 'w') as f:
    json.dump(reproducibility_info_ensemble, f, indent=2)

print("🎲 INFORMACIÓN DE REPRODUCTIBILIDAD DEL CONJUNTO PERSISTENTE:")
print(f"   Experimento: {reproducibility_info_ensemble['experimento']}")
print(f"   Número de modelos: {len(semillas)}")
print(f"   Semillas utilizadas: {semillas}")
print(f"   Semilla para Optuna: 42")
print(f"   Mejor modelo: #{mejor_modelo_idx + 1} (semilla {mejor_semilla})")
print(f"   MAE promedio: {df_metricas['mae'].mean():.4f} (±{df_metricas['mae'].std():.4f})")
print(f"   Modelos guardados en: {modelos_dir}/")

print(f"\n✅ Información de reproductibilidad guardada en:")
print(f"   {archivo_reproducibilidad}")

print(f"\n📊 RESUMEN FINAL DEL EXPERIMENTO CON PERSISTENCIA:")
print(f"   🔬 Estrategia: Conjunto de {len(semillas)} modelos LightGBM persistentes")
print(f"   ⭐ Tecnología: LINEAR_TREE + OPTUNA + 36 LAGS + PERSISTENCIA")
print(f"   🎯 Objetivo: Máxima robustez, precisión y reutilización")
print(f"   📁 Archivos: 6 archivos CSV + 20 modelos .pkl + 1 JSON reproductibilidad")
print(f"   🏆 Resultado: Sistema completo de ensemble reutilizable")
print(f"   🔄 Capacidad: Cargar y usar cualquier modelo individual por semilla")
print(f"   🎛️ Flexibilidad: Crear ensembles personalizados sin re-entrenar")

print(f"\n🎉 ¡SISTEMA DE MACHINE LEARNING PERSISTENTE COMPLETADO EXITOSAMENTE!")
print(f"💪 Máxima robustez, flexibilidad y capacidad de experimentación")

🎲 INFORMACIÓN DE REPRODUCTIBILIDAD DEL CONJUNTO PERSISTENTE:
   Experimento: LGBM_v3_FE_02_v3_linear_tree_opt_03_36lags_semillerio_persistente
   Número de modelos: 20
   Semillas utilizadas: [42, 109, 113, 151, 167, 179, 193, 199, 211, 241, 263, 281, 307, 331, 367, 401, 439, 487, 563, 617]
   Semilla para Optuna: 42
   Mejor modelo: #3 (semilla 113)
   MAE promedio: 9.1058 (±0.4433)
   Modelos guardados en: data/modelos_entrenados/sem_1_e3/

✅ Información de reproductibilidad guardada en:
   data/reproducibility_info_lgbm_v3_FE_02_v3_linear_tree_opt_36lags_sem_1_ensemble_persistente_e2.json

📊 RESUMEN FINAL DEL EXPERIMENTO CON PERSISTENCIA:
   🔬 Estrategia: Conjunto de 20 modelos LightGBM persistentes
   ⭐ Tecnología: LINEAR_TREE + OPTUNA + 36 LAGS + PERSISTENCIA
   🎯 Objetivo: Máxima robustez, precisión y reutilización
   📁 Archivos: 6 archivos CSV + 20 modelos .pkl + 1 JSON reproductibilidad
   🏆 Resultado: Sistema completo de ensemble reutilizable
   🔄 Capacidad: Cargar y usar cual